In [ ]:
import torch
import torchvision
from torch.nn import Sequential,Linear,ReLU,Tanh,Sigmoid,L1Loss,MSELoss,Softmax,CrossEntropyLoss
criterion = CrossEntropyLoss()
from torch.optim import Adam
import torchvision.datasets as dataset
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import DataLoader,TensorDataset
import numpy as np


In [ ]:
#enable cuda
device = 'cpu' if torch.cuda.is_available() else 'cuda:0'
device

'cpu'

In [ ]:
#load data
train_data = dataset.MNIST(root='./data',train=True,transform =transforms.ToTensor(),download=True )
test_data = dataset.MNIST(root='./data',train=False,transform =transforms.ToTensor(),download=True )
train_loader = DataLoader(dataset = train_data,batch_size=32,shuffle=True)
test_loader = DataLoader(dataset = test_data,batch_size=32,shuffle=True)
input_size = 28*28
print(train_data.classes)

['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']


In [ ]:
#784 --> 128 ---> 64 --> 36--->36 ---> 64 ---> 128 ---> 784
class denoise_auto_encoder(torch.nn.Module):
  def __init__(self):
    super().__init__()
  
  
    self.encoder = Sequential(
          Linear(input_size,128),
          ReLU(),
          Linear(128,64),
          ReLU(),
          Linear(64,36)
          
      )
    self.decoder = Sequential(
          Linear(36,64),
          ReLU(),
          Linear(64,128),
          ReLU(),
          Linear(128,input_size),
          Sigmoid()
      )

  def forward(self,input):
    enc= self.encoder(input)
    dec = self.decoder(enc)
    return enc,dec


In [ ]:
def plot_loss(loss):
  
  plt.style.use('fivethirtyeight')
  plt.xlabel('Epochs')
  plt.ylabel('Loss') 
  
  plt.plot(loss[-100:])


def train_denoising(model,optimizer,epochs,loss_fun,data_loader,input_size):
  
  for epoch in range(epochs):
    try:
      noise_factor=0.2
      epoch_loss=[]
      i=0
      for (image, label) in data_loader:
        image = image.reshape(-1, input_size)
        ## add random noise to the input images
        noisy_imgs = image + noise_factor * torch.randn(*image.shape)
        # Clip the images to be between 0 and 1
        noisy_imgs = np.clip(noisy_imgs, 0., 1.)

        image = noisy_imgs.to(device)        
        encoder_output,decoder_output = model(image)
        if i==0:
          encoder_output_list = encoder_output.detach().numpy()
          label_list = label.detach().numpy()
        else:
          encoder_output_list = np.append(encoder_output_list,encoder_output,axis=0)
          label_list = np.append(label_list,label)

        # Calculating the loss function
        loss = loss_fun(decoder_output, image)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Storing the losses in a list for plotting
        losses.append(loss)
        epoch_loss.append(loss)
        if epoch+1 == epochs: 
          output.append((epochs, image, decoder_output))
        print('epoch [{}], loss:{:.4f}'
            .format(epoch + 1, np.mean(epoch_loss)))
        print('entered try')
        scalled_encoder = StandardScaler().fit_transform(encoder_output_list)

      
        tsne = TSNE(n_components=2,metric='euclidean',random_state=100,
                    perplexity=np.sqrt(np.round(len(scalled_encoder),0)),learning_rate=10,
                    n_iter=1000,n_iter_without_progress=300,
                    min_grad_norm=1e-7,n_jobs=-1) 
      
      tsne_embedded = tsne.fit_transform(scalled_encoder)  
      return model,tsne_embedded,encoder_output_list,decoder_output,label_list
    except Exception as e: print(e)

    # finally:
    #   plot_loss(losses)
    #   return model
      


#extracting encoer and adding classification layer
class classifier(torch.nn.Module):
  def __init__(self):
    super().__init__()
    
    #Encoder layers
    self.layer = Sequential(
        Linear(36,10),
        Softmax(1)
    )
   
  def forward(self, x):
    #Forward prop
    x = self.layer(x)
    return x


In [ ]:

mae_loss = MSELoss()

model = denoise_auto_encoder().cuda()
optimizer = Adam(model.parameters(),lr = .01,weight_decay =1e-5)
epochs =10
output = []
losses = []
model,tsme_emdeded,encoder_train,decoder_train,labels = train_denoising(model,optimizer,epochs,mae_loss,train_loader,input_size)

ax1 = sns.scatterplot(x=tsme_emdeded[:,0],y=tsme_emdeded[:,1],hue=labels)
ax1.set(xlabel='tsne-0',ylabel='tsne-1',title='tsne-0 vs tsne-1')




Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_addmm)
Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_addmm)
Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_addmm)
Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_addmm)
Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_addmm)
Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_addmm)
Expected all tensors to be o

TypeError: ignored

In [ ]:
def train_classifier(model,optimizer,epochs,loss_fun,data_loader,input_size):
  for epoch in range(epochs):
    try:
      epoch_loss=[]
      i=0
      for (image, label) in data_loader:
        image = image.reshape(-1, input_size)
        ## add random noise to the input images
        image = image.to(device)        
        output = model(image)
        # if i==0:
        #   encoder_output_list = output.detach().numpy()
        #   label_list = label.detach().numpy()
        # else:
        #   encoder_output_list = np.append(encoder_output_list,output,axis=0)
        #   label_list = np.append(label_list,label)

        # Calculating the loss function
        loss = loss_fun(output, label.long())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Storing the losses in a list for plotting
        losses.append(loss)
        epoch_loss.append(loss)
        if epoch+1 == epochs: 
          output.append((epochs, image, decoder_output))
        print('epoch [{}], loss:{:.4f}'
            .format(epoch + 1, np.mean(epoch_loss)))
        print('entered try')
    return model
    except Exception as e: print(e)


In [ ]:
tensor_x = torch.Tensor(encoder_train) # transform to torch tensor
tensor_y = torch.Tensor(labels)
new_dataset = TensorDataset(tensor_x,tensor_y) 
classifier_loader = DataLoader(new_dataset, batch_size=32) 
criterion = CrossEntropyLoss()
clasifier_model = classifier()

model = train_classifier(clasifier_model,optimizer,epochscriterion,classifier_loader,input_size)